In [4]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import json
import tqdm

# read xlsx file
df = pd.read_excel(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\0.external_data\FAO_price_data\FAO_DATA_FOR_MIGUEL.xlsx')

new_df = df[['long', 'lat']]

# remove duplicates
new_df = new_df.drop_duplicates()
#reset index
new_df = new_df.reset_index(drop=True)
# make API request based on Longtiude and Latitude


for i in tqdm.tqdm(range(len(new_df))):
    url = 'https://api.opentopodata.org/v1/aster30m?locations={},{}'.format(new_df['lat'][i], new_df['long'][i])
    response = requests.get(url)
    data = json.loads(response.text)
    # create a new column to store the elevation data
    if 'results' in data:
        new_df.loc[i, 'elevation'] = data['results'][0]['elevation']



100%|██████████| 421/421 [01:27<00:00,  4.80it/s]


In [5]:
for i in tqdm.tqdm(range(len(new_df))):
   url = 'https://rest.isric.org/soilgrids/v2.0/properties/query?lon={}&lat={}&property=bdod&depth=0-5cm&value=mean'.format(new_df['long'][i], new_df['lat'][i])
   response = requests.get(url)
   data = json.loads(response.text)
   if 'properties' in data:
      new_df.loc[i, 'soil'] = data['properties']['layers'][0]['depths'][0]['values']['mean']

# merge the elevation data with the original dataframe
df = pd.merge(df, new_df, on=['long', 'lat'], how='left')
#remove elevation_x and rename elevation_y to elevation
df = df.drop(['elevation_x'], axis=1)
df = df.rename(columns={'elevation_y': 'elevation'})


  0%|          | 0/421 [00:00<?, ?it/s]


IndexError: list index out of range

In [ ]:
# output the merged dataframe to a csv file
df.to_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\\2.merge_data\FAO_elevation_soil.csv', index=False)